In [1]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Input, Flatten, Dropout
from keras.layers import LSTM, Embedding,GRU
from keras.models import Sequential

In [10]:
import pandas as pd
import numpy as np
pd_ori=pd.read_csv('all.csv')
from sklearn.model_selection import StratifiedShuffleSplit
sss= StratifiedShuffleSplit(n_splits=1, test_size=0.999, random_state=42)
for train_index, test_index in sss.split(pd_ori, pd_ori['label']):
    set_1=pd_ori.iloc[train_index]
    set_2=pd_ori.iloc[test_index]
set_2['label'].value_counts()/len(set_2)


set_2.reset_index(inplace=True)
sss= StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in sss.split(set_2, set_2['label']):
    train_df=set_2.iloc[train_index]
    devtest_df=set_2.iloc[test_index]
train_index=train_df.index.values.tolist()


sss= StratifiedShuffleSplit(n_splits=1, test_size=0.5, random_state=42)
for train_index, test_index in sss.split(devtest_df, devtest_df['label']):
    dev_df=devtest_df.iloc[train_index]
    test_df=devtest_df.iloc[test_index]
test_index=test_df.index.values.tolist()
dev_index=dev_df.index.values.tolist()



In [11]:
len(set_2)

607990

In [12]:
all_texts=set_2['review'].tolist()
all_labels=set_2['label'].tolist()
MAX_SEQUENCE_LENGTH = 500    #the max seqeunce length
EMBEDDING_DIM = 200    #the size of embedding
tokenizer = Tokenizer()
tokenizer.fit_on_texts(all_texts)
sequences = tokenizer.texts_to_sequences(all_texts)
word_index = tokenizer.word_index
print('There are unique tokens.' % len(word_index))
data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
labels = to_categorical(np.asarray(all_labels))
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)
X_train=data[train_index]
y_train=labels[train_index]
X_test=data[test_index]
y_test=labels[test_index]
X_dev=data[dev_index]
y_dev=labels[dev_index]


Found 204488 unique tokens.
Shape of data tensor: (607990, 500)
Shape of label tensor: (607990, 2)


In [13]:
len(X_dev)

60799

In [14]:
model = Sequential()
model.add(Embedding(len(word_index) + 1, EMBEDDING_DIM, input_length=MAX_SEQUENCE_LENGTH))
model.add(LSTM(200, dropout=0.2, recurrent_dropout=0.2))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dense(labels.shape[1], activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 500, 200)          40897800  
_________________________________________________________________
lstm (LSTM)                  (None, 200)               320800    
_________________________________________________________________
dropout (Dropout)            (None, 200)               0         
_________________________________________________________________
dense (Dense)                (None, 64)                12864     
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 130       
Total params: 41,231,594
Trainable params: 41,231,594
Non-trainable params: 0
_________________________________________________________________


In [16]:
model.compile(loss='binary_crossentropy',
                  optimizer='rmsprop',
                  metrics=['acc'])
print(model.metrics_names)
model.fit(X_train, y_train, validation_data=(X_dev, y_dev), epochs=2, batch_size=128)
model.save('lstm.h5')

[]
Epoch 1/2
475/475 [==============================] - 2789s 6s/step - loss: 0.3956 - acc: 0.8636 - val_loss: 0.3734 - val_acc: 0.8678
Epoch 2/2
475/475 [==============================] - 2791s 6s/step - loss: 0.3646 - acc: 0.8650 - val_loss: 0.3639 - val_acc: 0.8679


In [19]:
model.evaluate(X_test, y_test)

1900/1900 [==============================] - 507s 267ms/step - loss: 0.3624 - acc: 0.8680


[0.36241376399993896, 0.8680077195167542]

In [20]:
y_pred=model.predict(X_test)

In [23]:
y_pred

array([[0.9973808 , 0.00261924],
       [0.99802303, 0.001977  ],
       [0.9666526 , 0.03334745],
       ...,
       [0.9184121 , 0.08158792],
       [0.99322444, 0.00677551],
       [0.8207592 , 0.17924082]], dtype=float32)

In [24]:
from sklearn.metrics import roc_auc_score

In [26]:
print(roc_auc_score(y_test, y_pred))

0.7100465386691508
